In [19]:
import torch
from transformer_lens import HookedTransformer, utils
import tqdm
from e2e_sae import SAETransformer

In [14]:
model_id = "h9hrelni"
text = "sparsify/gpt2/" + model_id
sae_transformer_model = SAETransformer.from_wandb(text)

model = sae_transformer_model.tlens_model
sae = sae_transformer_model.saes[f"blocks-6-hook_resid_pre"]

Loaded pretrained model gpt2-small into HookedTransformer


In [2]:
token_tensor = torch.load("../token_tensor.pt")

In [31]:
token_tensor.shape

torch.Size([16000, 128])

In [24]:
batch_size = 64

dataloader = torch.utils.data.DataLoader(
    token_tensor,
    batch_size=batch_size,
    shuffle=False
)

hook_loc = "resid_pre"
layer = 6

activation_loc = utils.get_act_name(hook_loc, layer)

In [30]:
# get all activations

first = True

for ix, batch_tokens in enumerate(tqdm.tqdm(dataloader)):
    with torch.inference_mode():
        _, cache = model.run_with_cache(
            batch_tokens, 
            prepend_bos=True,
            names_filter=[activation_loc]
        )
        activations = cache[activation_loc]

        if first:
            all_activations = activations
            first = False
        else:
            all_activations = torch.cat((all_activations, activations))

  1%|█▊                                                                                                                                                  | 3/250 [02:31<3:27:58, 50.52s/it]


KeyboardInterrupt: 

In [27]:
activations = cache[activation_loc]

In [29]:
activations.shape

torch.Size([64, 128, 768])

In [ ]:
for ix, batch_tokens in enumerate(tqdm.tqdm(dataloader)):
    with torch.inference_mode():
        _, cache = model.run_with_cache(
            batch_tokens, 
            prepend_bos=True,
            names_filter=[activation_loc]
        )
        activations = cache[activation_loc]
        if hook_loc == "z":
            activations = einops.rearrange(
                activations, "batch seq n_heads d_head -> batch seq (n_heads d_head)",
            )

        if e2e:
            sae_out, feature_acts = sae(activations) # for E2E SAEs
        else:
            sae_out, feature_acts, _, _, _, _ = sae(activations)
        
        ablation_hooks = create_ablation_hooks(sae_out, pos=pos)
        
        if hook_loc == "z":
            ablation_hooks = [
                (name, partial(attn_hook_wrapper, hook_fn=hook_fn))
                for name, hook_fn in ablation_hooks
            ]
        
        batch_result_df = run_all_ablations(model, batch_tokens, ablation_hooks, layer=layer, hook_loc=hook_loc)
        
        l0 = (feature_acts > 0).float().sum(dim=-1).cpu().numpy()[:, :-1].flatten()
        l1 = feature_acts.abs().sum(dim=-1).cpu().numpy()[:, :-1].flatten()
        reconstruction_error = (activations - sae_out).norm(dim=-1).cpu().numpy()[:, :-1].flatten()
        
        batch_result_df['sae_l0'] = l0
        batch_result_df['sae_l1'] = l1
        batch_result_df['reconstruction_error'] = reconstruction_error
        batch_result_df['norm'] = activations.norm(dim=-1).cpu().numpy()[:, :-1].flatten()
        batch_result_df['sae_norm'] = sae_out.norm(dim=-1).cpu().numpy()[:, :-1].flatten()
        batch_result_df['cos'] = cos_sim(activations, sae_out).cpu().numpy()[:, :-1].flatten()
        
        result_dfs.append(batch_result_df)